 # Práctica  5.
 ----
  
  Universidad : UTEC \\
  Curso       : Inteligencia Artificial \\
  Profesor    : Cristian López Del Alamo \\
  Tema        : Árboles de Decisión \\
  

 ----

 Nombres y  Apellidos de Integrantes: (No olvide poner el % de participacion)
 - Integrante 1: Juan Sebastian Sara 100%
 - Integrante 2: Vasco Diaz Hurtado 100%
 - Integrante 3: Marcela Espinoza 100%
 - Integrante 4: Yamileth Rincón 100%

*Una vez concluya la práctica debe subir el link de su colab a este  [Drive]
(https://docs.google.com/spreadsheets/d/1XCxGVmf8g29C7RZSPOqxvZHjjRZg45LHWR8mkZOfS1o/edit?usp=drivesdk)*


Para este ejercicio, usted deberá contruir su propio árbol de desición.
- Tenga en cuenta que se trata de un algoritmo recursivo.
- El caso base se da cuanto todos los elementos de un nodo tiene las mismas etiquetas, es decir, es un nodo terminal. Luego, el label de ese nodo toma el valor de la etiqueta común.
- En el caso que no sea un nodo terminal, el algoritmo debe buscar uno de los
feactures por el cual dividirse y para esto use Ganacia de Información (Entropía o Gini).
- Divida el dataset usando el feacture que genere una mayor ganancia de información en el padre o un menor GINI y llame recursivamente a la función create_DT.

Usted usará la base de datos iris, con 4 características y 3 clases.
Tome aleatoriamente 80% de los datos para crear el árbol y el resto para
probar el accuracy de la predicción.
Finalmente, muestra mediante una matriz de confusión el **accuracy** de su modelo.

Trabaje en equipo:

[Link de apoyo 1](https://towardsdatascience.com/the-simple-math-behind-3-decision-tree-splitting-criterions-85d4de2a75fe)

[Link de apoyo 2](https://www.quantstart.com/articles/Beginners-Guide-to-Decision-Trees-for-Supervised-Machine-Learning/)



In [ ]:
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

iris = sns.load_dataset('iris')
print(iris.head())


class Nodo:
    def __init__(self, X, Y, index=None):
        self.X = X
        self.Y = Y
        self.index = index
        self.split_feature = None
        self.split_value = None
        self.left = None
        self.right = None
        self.label = None

    def IsTerminal(self):
        return len(np.unique(self.Y)) == 1

    def Entropy(self, Y):
        labels, counts = np.unique(Y, return_counts=True)
        probs = counts / len(Y)
        entropy = - np.sum(probs * np.log2(probs))
        return entropy

    def BestSplit(self):
        best_gain = 0
        best_feature = None
        best_value = None

        current_entropy = self.Entropy(self.Y)

        for feature in self.index:
            unique_values = np.unique(self.X[feature])
            for value in unique_values:
                left_Y = self.Y[self.X[feature] <= value]
                right_Y = self.Y[self.X[feature] > value]
                left_prob = len(left_Y) / len(self.Y)
                right_prob = 1 - left_prob
                new_entropy = left_prob * self.Entropy(left_Y) + right_prob * self.Entropy(right_Y)
                gain = current_entropy - new_entropy

                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_value = value

        self.split_feature = best_feature
        self.split_value = best_value

        return best_feature, best_value


class DT:
    def __init__(self):
        self.m_Root = None

    def create_DT(self, X, Y, index=None):
        if index is None:
            index = X.columns.tolist()

        node = Nodo(X, Y, index)

        if node.IsTerminal():
            node.label = Y.iloc[0]
            return node

        feature, value = node.BestSplit()

        if feature is None:
            node.label = Y.value_counts().idxmax()
            return node

        node.left = self.create_DT(X[X[feature] <= value], Y[X[feature] <= value], index)
        node.right = self.create_DT(X[X[feature] > value], Y[X[feature] > value], index)

        return node

    def predict(self, node, x):
        if node.label is not None:
            return node.label

        if x[node.split_feature] <= node.split_value:
            return self.predict(node.left, x)
        else:
            return self.predict(node.right, x)

    def fit(self, X, Y):
        self.m_Root = self.create_DT(X, Y)


X = iris.drop('species', axis=1)
Y = iris['species']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

tree = DT()
tree.fit(X_train, Y_train)

predictions = X_test.apply(lambda x: tree.predict(tree.m_Root, x), axis=1)
print(confusion_matrix(Y_test, predictions))
print('Accuracy:', accuracy_score(Y_test, predictions))


   sepal_length  sepal_width  petal_length  petal_width species
0           5.1          3.5           1.4          0.2  setosa
1           4.9          3.0           1.4          0.2  setosa
2           4.7          3.2           1.3          0.2  setosa
3           4.6          3.1           1.5          0.2  setosa
4           5.0          3.6           1.4          0.2  setosa
[[10  0  0]
 [ 0  9  0]
 [ 0  0 11]]
Accuracy: 1.0
